<a href="https://colab.research.google.com/github/vlgul/series-de-tiempo/blob/main/Capitulo_8_Tema_8_4_M%C3%A9todo_aditivo_de_Holt_Winters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Capitulo 8:  Suavización exponencial** ###

### 8.3. Método aditivo de Holt-Winters ### 
 
 Se presentan las ventas trimestrales de los cuatro años anteriores de una tienda de bicicletas en Suiza. La gráfica sugiere que las ventas de la bicicleta muestran una demanda lineal y una variación constante (aditiva). Por lo tanto, aplicamos el método de Holt-Winters a los datos para dar con los pronósticos de las ventas futuras de la bicicleta de montaña.

 Variables:
 Lt: Nivel de estimación
 Bt: Tasa de crecimiento.
 Snt: Factor estacional.
 Pt: Pronóstico hecho en el último periodo. 
 Et: Error de pronóstico
 Et2: Error cuadrado del pronóstico
 Yest: Estimaciónes de regresión 
 St: Información sin tendencia
 Sn_3, Sn_2, Sn_1, S0: Promedio de factores estacionales con L=4 factores.
 Mn: Vector donde se almacenan los valores de los factores estacionales promedio.


In [1]:
n<-17
p<-5
Lt<-double(n)
Bt<-double(n)
Et<-double(n)
Et2<-double(n)
Pt<-double(n)
St<-double(n)
Snt<-double(n)
Mn<-double(n)
Yest<-double(n)

Agregar datos de ventas de bicicletas de montaña del periodo 0 al 16

In [2]:
t<-seq(0,16)
Yt<-c(0,10,31,43,16,11,33,45,17,14,36,50,21,19,41,55,25)

Ajustar una recta de tendencia de minimos cuadrados a la mitad de los datos totales n=16 y extraer valores para estimación inicial de los coeficientes para asignar a L0 y B0.

In [3]:
f <- lm(Yt[2:17] ~ t[2:17])
Reg<-summary(f)
L0<-Reg$coefficients[1]
B0<-Reg$coefficients[2]

Agregar valor inicial de coeficientes al nivel, la tasa de crecimiento y definir constantes de suavizacion alfa=0.2, gamma=0.1 y delta=0.1

In [4]:
Lt[1]<-L0
Bt[1]<-B0
alfa<-0.2
gamma<-0.1 
delta<-0.1

Calcular las estimaciones de regresión sin tendencia

In [5]:
for (i in 2:n){
  Yest[i]<-Lt[1]+Bt[1]*(i-1)
  St[i]<-Yt[i]-Yest[i]
}

Calcular Factor estacional para L = 4, Sn_3, Sn_2, Sn_1, Sn0 y guardar valores en el vector Mn que almacene la información replicada para el cálculo de Pt.

In [6]:
Sn_3<-((Yt[2]-Yest[2])+(Yt[6]-Yest[6])+(Yt[10]-Yest[10])+(Yt[14]-Yest[14]))/4
Sn_2<-((Yt[3]-Yest[3])+(Yt[7]-Yest[7])+(Yt[11]-Yest[11])+(Yt[15]-Yest[15]))/4
Sn_1<-((Yt[4]-Yest[4])+(Yt[8]-Yest[8])+(Yt[12]-Yest[12])+(Yt[16]-Yest[16]))/4
S0<-((Yt[5]-Yest[5])+(Yt[9]-Yest[9])+(Yt[13]-Yest[13])+(Yt[17]-Yest[17]))/4
Mn[seq (1,n,by=4)]<-Sn_3
Mn[seq (2,n,by=4)]<-Sn_2
Mn[seq (3,n,by=4)]<-Sn_1
Mn[seq (4,n,by=4)]<-S0
Snt[1]<-S0

Generar el cálculo de datos respecto a las variables Lt, Bt, y Snt, para t>2, con los errores cuadráticos.

In [7]:
for (i in 2:n){
  Pt[i]<-Lt[(i-1)]+Bt[(i-1)]+Mn[i-1]
  Lt[i]<-Lt[(i-1)]+Bt[(i-1)]+(alfa*(Yt[i]- Pt[i]))
  Bt[i]<-Bt[(i-1)]+(alfa*gamma)*(Yt[i]-Pt[i])
  Snt[i]<-Mn[i-1]+((1-alfa)*delta*(Yt[(i)]-Pt[i]))
  Et[i]<-Yt[i]-Pt[i]
  Et2[i]<-Et[i]^2
}

Visualizar los valores resultantes en formato de tabla de Fig. 8.9

In [8]:
Bicicletas<-data.frame(t,Yt,Lt,Bt,Snt,Pt,Et,Et2,Yest,St,Mn)
Bicicletas

t,Yt,Lt,Bt,Snt,Pt,Et,Et2,Yest,St,Mn
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,20.85000,0.9808824,-10.908824,0.000000,0.0000000,0.00000000,0.00000,0.000000,-14.216176
1,10,22.30794,1.0285882,-14.025353,7.614706,2.3852941,5.68962803,21.83088,-11.830882,6.552941
2,31,23.55864,1.0507988,6.641784,29.889471,1.1105294,1.23327557,22.81176,8.188235,18.572059
3,43,24.57314,1.0471690,18.557539,43.181493,-0.1814929,0.03293969,23.79265,19.207353,-10.908824
4,16,25.87801,1.0729393,-10.805742,14.711481,1.2885190,1.66028130,24.77353,-8.773529,-14.216176
5,11,26.60399,1.0382439,-14.354958,12.734771,-1.7347712,3.00943103,25.75441,-14.754412,6.552941
6,33,27.40320,1.0143404,6.457327,34.195179,-1.1951785,1.42845167,26.73529,6.264706,18.572059
7,45,28.01962,0.9745483,18.412891,46.989601,-1.9896008,3.95851137,27.71618,17.283824,-10.908824
8,17,28.77710,0.9528414,-10.995651,18.085347,-1.0853466,1.17797730,28.69706,-11.697059,-14.216176


Valor del error estandar SSE para calcular el error de pronostico S

In [9]:
t<-16
SSE<-sum(Et2)
S<-sqrt(SSE/(t-2))

Reproducir los valores resultantes en formato de tabla de Fig. 8.10
con los valores nuevos para alfa<-0.561, gamma<-0.0, delta1<-0.0.

In [10]:
alfa1<-0.561
gamma1<-0.0
delta1<-0.0
for (i in 2:n){
  Pt[i]<-Lt[(i-1)]+Bt[(i-1)]+Mn[i-1]
  Lt[i]<-Lt[(i-1)]+Bt[(i-1)]+(alfa1*(Yt[i]- Pt[i]))
  Bt[i]<-Bt[(i-1)]+(alfa1*gamma1)*(Yt[i]-Pt[i])
  Snt[i]<-Mn[i-1]+((1-alfa1)*delta1*(Yt[(i)]-Pt[i]))
  Et[i]<-Yt[i]-Pt[i]
  Et2[i]<-Et[i]^2
}

Visualizar los valores resultantes en formato de tabla de Fig. 8.10.  Generar intervalos de prediccion del 95% para Yest=17,18,19 Z=1.96

Para Yest=17 para un intervalo de predicción del 95%:

In [11]:
z<-qnorm(0.975, mean = 0, sd = 1)
Yest17<-Lt[16]+Bt[16]+Mn[13]
Y17i<-Yest17-z*S*sqrt(1)
Y17s<-Yest17+z*S*sqrt(1)
cat("Valor Inferior =",Y17i,"\n")
cat("Valor Superior =",Y17s)

Valor Inferior = 20.10834 
Valor Superior = 25.25274

Para Yest=18 para un intervalo de predicci?n del 95%

In [12]:
Yest18<-Lt[16]+2*Bt[16]+Mn[14]
Y18i<-Yest18-z*S*sqrt(1+(alfa1^2)*(1+gamma1)^2)
Y18s<-Yest18+z*S*sqrt(1+(alfa1^2)*(1+gamma1)^2)
cat("Valor Inferior =",Y18i,"\n")
cat("Valor Superior =",Y18s)

Valor Inferior = 41.48122 
Valor Superior = 47.37986

 Para Yest=19 para un intervalo de predicción del 95%

In [13]:
Yest19<-Lt[16]+3*Bt[16]+Mn[15]
Y19i<-Yest19-z*S*sqrt(1+(alfa1^2)*((1+gamma1)^2)+(alfa1^2)*(1+2*gamma1)^2)
Y19s<-Yest19+z*S*sqrt(1+(alfa1^2)*((1+gamma1)^2)+(alfa1^2)*(1+2*gamma1)^2)
cat("Valor Inferior =",Y19i,"\n")
cat("Valor Superior =",Y19s)

Valor Inferior = 54.14714 
Valor Superior = 60.71395